In [1]:
import pandas as pd
books = pd.read_csv("books_with_categories.csv")

In [3]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None)
classifier("I love this!")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528691716492176},
  {'label': 'neutral', 'score': 0.005764589179307222},
  {'label': 'anger', 'score': 0.004419791977852583},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.0004138525982853025}]]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [4]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [ ]:
classifier(books["description"][0]) #classify the whole book description

[[{'label': 'fear', 'score': 0.6548399925231934},
  {'label': 'neutral', 'score': 0.1698525995016098},
  {'label': 'sadness', 'score': 0.11640939861536026},
  {'label': 'surprise', 'score': 0.02070068009197712},
  {'label': 'disgust', 'score': 0.019100721925497055},
  {'label': 'joy', 'score': 0.015161462128162384},
  {'label': 'anger', 'score': 0.003935154061764479}]]

In [6]:
#split description into individual sentences and classify each sentence
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296021580696106},
  {'label': 'neutral', 'score': 0.14038598537445068},
  {'label': 'fear', 'score': 0.06816227734088898},
  {'label': 'joy', 'score': 0.04794260486960411},
  {'label': 'anger', 'score': 0.009156367741525173},
  {'label': 'disgust', 'score': 0.0026284768246114254},
  {'label': 'sadness', 'score': 0.002122163772583008}],
 [{'label': 'neutral', 'score': 0.4493706524372101},
  {'label': 'disgust', 'score': 0.27359113097190857},
  {'label': 'joy', 'score': 0.1090831384062767},
  {'label': 'sadness', 'score': 0.09362747520208359},
  {'label': 'anger', 'score': 0.04047833010554314},
  {'label': 'surprise', 'score': 0.02697019837796688},
  {'label': 'fear', 'score': 0.006879049818962812}],
 [{'label': 'neutral', 'score': 0.6462159156799316},
  {'label': 'sadness', 'score': 0.242733433842659},
  {'label': 'disgust', 'score': 0.043422672897577286},
  {'label': 'surprise', 'score': 0.02830057218670845},
  {'label': 'joy', 'score': 0.01421149075

In [7]:
#see if sentence tone matches classification
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [13]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [14]:
predictions[3]

[{'label': 'fear', 'score': 0.9281682968139648},
 {'label': 'anger', 'score': 0.03219081461429596},
 {'label': 'neutral', 'score': 0.012808660045266151},
 {'label': 'sadness', 'score': 0.008756875991821289},
 {'label': 'surprise', 'score': 0.008597892709076405},
 {'label': 'disgust', 'score': 0.008431807160377502},
 {'label': 'joy', 'score': 0.0010455832816660404}]

In [16]:
#get max score for each emotion in the book description
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [17]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

emotion_scores

{'anger': [np.float64(0.06413363665342331),
  np.float64(0.6126194596290588),
  np.float64(0.06413363665342331),
  np.float64(0.35148516297340393),
  np.float64(0.0814124271273613),
  np.float64(0.23222433030605316),
  np.float64(0.5381842255592346),
  np.float64(0.06413363665342331),
  np.float64(0.3006698489189148),
  np.float64(0.06413363665342331)],
 'disgust': [np.float64(0.27359113097190857),
  np.float64(0.3482842445373535),
  np.float64(0.10400673747062683),
  np.float64(0.1507222205400467),
  np.float64(0.18449550867080688),
  np.float64(0.7271752953529358),
  np.float64(0.15585486590862274),
  np.float64(0.10400673747062683),
  np.float64(0.27948102355003357),
  np.float64(0.1779264211654663)],
 'fear': [np.float64(0.9281682968139648),
  np.float64(0.942527711391449),
  np.float64(0.9723207950592041),
  np.float64(0.36070549488067627),
  np.float64(0.09504342079162598),
  np.float64(0.051362838596105576),
  np.float64(0.7474275231361389),
  np.float64(0.4044971764087677),
  n

In [ ]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

 64%|██████▍   | 3321/5197 [03:00<01:43, 18.16it/s]

In [ ]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn
emotions_df

In [ ]:
books = pd.merge(books, emotions_df, on = "isbn13")
books

In [ ]:
books.to_csv("books_with_emotions.csv", index = False)